In [1]:
import pandas as pd
import numpy as np
import json
import datetime 

In [80]:
sourceCheckins = pd.read_csv('data/July/server/checkins.tsv', sep='\t')
checkins = pd.read_csv('data/July/server/extendedCheckinsHistory.tsv', sep='\t')
checkins1 = pd.read_csv('data/July/server/extendedCheckinsHistoryJune.tsv', sep='\t')
checkpoints = pd.read_csv('data/July/server/checkPoints.tsv', sep='\t')

In [81]:
len(np.unique(checkins1.userId.tolist()))

333

In [92]:
allCheckins = allCheckins.drop_duplicates(subset = ['checkinId'])

In [89]:
allCheckins = pd.concat([checkins1, checkins])

In [94]:
allCheckins.to_csv('data/July/server/currentExtendedCheckinsHistory.tsv', sep='\t', encoding='utf-8')

In [118]:
sortedVenues = []
sortedVenuesIds = []
for venueId, group in allCheckins.groupby('venueId').agg('size').order(ascending = False).iteritems():
    sortedVenues.append({'venueId':venueId,
                        'group':group})
    sortedVenuesIds.append(venueId)

In [91]:
allCheckins[:2]

,Unnamed: 0,Unnamed: 0.1,categoryId,categoryName,checkinId,checkinsCount,createdAt,createdAtDate,lat,lng,name,tipCount,userId,usersCount,venueId
0,0,0,4bf58dd8d48988d1fd931735,Metro Station,558ac442498e91c19449d814,14585,1435157570,2015-06-24 17:52:50,55.782920,37.720699,Метро Семёновская (metro Semyonovskaya),26,74434491,3460,4bed6c2c9868a5933c555d46
1,1,1,4bf58dd8d48988d1d3941735,Vegetarian / Vegan Restaurant,558ababc498e7aa1a83dfb16,10096,1435155132,2015-06-24 17:12:12,55.758369,37.657153,Джаганнат,222,74434491,4972,4d8f05153bfef04d84a6a586


In [99]:
sortedVenues[:10]

[{'group': 595, 'venueId': '4bdd5f27f219c9b6bb751210'},
 {'group': 411, 'venueId': '4bfc9df283bbd13aa4d462c5'},
 {'group': 263, 'venueId': '4bc090ca920eb7132e14192c'},
 {'group': 227, 'venueId': '4e4aab0f62e13240b700e034'},
 {'group': 215, 'venueId': '4bc8760792b376b03767513a'},
 {'group': 213, 'venueId': '4b62c18df964a52069512ae3'},
 {'group': 193, 'venueId': '4b63406af964a520a36d2ae3'},
 {'group': 180, 'venueId': '4bb3345942959c74d79d212c'},
 {'group': 175, 'venueId': '520f117111d2a782f1211c3b'},
 {'group': 164, 'venueId': '53110e9be4b03443dd848438'}]

In [113]:
allCheckins[allCheckins.venueId == '520f117111d2a782f1211c3b'][:1]['name']

935    World Class Lite
Name: name, dtype: object

In [115]:
#'World Class Lite in top ten!, 520f117111d2a782f1211c3b'

In [142]:
groupedUsers = allCheckins.groupby('userId')
userVenues = []
for index, group in enumerate(groupedUsers):
        userVenues.append(group[1]['venueId'].tolist())

In [144]:
len(groupedUsers)

450

In [122]:
top50VenuesCheckins = allCheckins[allCheckins.venueId.isin(sortedVenuesIds[:50])]

In [127]:
for venueId in sortedVenuesIds[:50]:
    print top50VenuesCheckins[top50VenuesCheckins.venueId == venueId][:1]['name']

3    ЦПКиО им. Горького / Gorky Park
Name: name, dtype: object
485    Москва / Moscow
Name: name, dtype: object
348    Парк искусств «Музеон» / Muzeon Park
Name: name, dtype: object
639    ВДНХ (Выставка достижений народного хозяйства)
Name: name, dtype: object
187    ПКиО «Сокольники»
Name: name, dtype: object
1127    Международный аэропорт Домодедово / Domodedovo...
Name: name, dtype: object
317    ТРЦ «Европейский»
Name: name, dtype: object
807    Красная площадь / Red Square
Name: name, dtype: object
935    World Class Lite
Name: name, dtype: object
1094    Россия / Russia
Name: name, dtype: object
2    ТРК «Атриум» / Atrium Mall
Name: name, dtype: object
1335    Арбат / Arbat Street
Name: name, dtype: object
944    Музей-заповедник «Коломенское»
Name: name, dtype: object
630    Афимолл Сити / Afimall City
Name: name, dtype: object
901    Физика
Name: name, dtype: object
333    Нескучный сад
Name: name, dtype: object
687    Центральный детский магазин
Name: name, dtype: object
252 

In [ ]:
from pymining import itemmining
relim_input = itemmining.get_relim_input(userVenues)
report = itemmining.relim(relim_input, min_support=2)
report